In [9]:
#installing selenium
!pip install selenium
!pip install bs4
!pip install requests

In [10]:
#importing libraries and drivers
import selenium
import pandas as pd
import warnings
import numpy as np
import time
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException , NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
warnings.filterwarnings('ignore')

# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
#command to run chrome
driver = webdriver.Chrome(r'chromedriver.exe')

#get driver to visit defined URL
driver.get('https://amazon.in')
time.sleep(2)

# Taking input from user about product search
User_input=input('Enter the title of Product you are interest in search :')

Enter the title of Product you are interest in search :guitar


In [4]:
#input command in search bar
search=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')

#clearing any previous input in search bar
search.clear()

#feeding input specified by user to search menu through send keys
search.send_keys(User_input)

#finding Search button for clicking through xpath
search_button=driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]')

#clicking search button
search_button.click()

# Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
b_name=[]
product_name=[]
rating=[]
no_of_rating=[]
price=[]
ret_exchange=[]
exp_delivery=[]
availability=[]
other_details=[]
product_url=[]
urls=[]
for i in range(0,3):
    if i>2:
        break
    p_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in p_urls:
        urls.append(i.get_attribute('href'))


    for i in urls:
        driver.get(i)
        time.sleep(2)
    #scraping data for brand name
        try:
            bran=driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
            b_name.append(bran.text)
        except NoSuchElementException as e:
            b_name.append("--")

        #scraping data for product name
        try:
            prod=driver.find_element(By.XPATH,"//h1[@id='title']/span")
            product_name.append(prod.text)
        except NoSuchElementException as e:
            product_name.append("--")

        #scraping data for rating
        try:
            rat=driver.find_element(By.XPATH,"//span[@id='acrPopover']/span/a/i/span")
            rating.append(rat.text)
        except NoSuchElementException as e:
            rating.append("--")

        #scraping data for rating no.
        try:
            rat_no=driver.find_element(By.XPATH,"//span[@id='acrCustomerReviewText']")
            no_of_rating.append(rat_no.text)
        except NoSuchElementException as e:
            no_of_rating.append("--")

        #scraping data for price
        try:
            pri=driver.find_element(By.XPATH,"//div[@id='apex_desktop']/div/div/table/tbody/tr[2]/td[2]/span/span[2]")
            price.append(pri.text)
        except NoSuchElementException as e:
            price.append("--")

        #scaping data for return
        try:
            retu=driver.find_element(By.XPATH,"//div[@id='RETURNS_POLICY']/span/div[2]/a")
            ret_exchange.append(retu.text)
        except NoSuchElementException as e:
            ret_exchange.append("--")

        #scraping data for expected delivery
        try:
            deliv=driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']/b")
            exp_delivery.append(deliv.text)
        except NoSuchElementException as e:
            exp_delivery.append("--")

        #scraping data for availability
        try:
            avail=driver.find_element(By.XPATH,"//div[@id='availability']/span")
            availability.append(avail.text)
        except NoSuchElementException as e:
            availability.append("--")

        #scraping data for other details
        try:
            od=driver.find_element(By.XPATH,"//div[@id='feature-bullets']/ul/li/span")
            other_details.append(od.text)
        except NoSuchElementException as e:
            other_details.append("--")
     
    # selecting and clicking next page
    try:
        if i==1:
            n_button=driver.find_element(By.XPATH,"//div[@class='a-section a-spacing-none a-padding-base']/div/ul/li[3]/a")
            n_url=n_button.get_attribute('href')
            driver.get(n_url)
            time.sleep(2)
    except:
        if i==2:
            n_button=driver.find_element(By.XPATH,"//div[@class='a-section a-spacing-none a-padding-base']/div/ul/li[4]/a")
            n_url=n_button.get_attribute('href')
            driver.get(n_url)
            
#placing everything into dataframe
df=pd.DataFrame(list(zip(b_name,product_name,rating,no_of_rating,price,ret_exchange,exp_delivery,availability,other_details,urls)),columns=["Brand Name","Product","Rating","No. of rating","Price","Return/Exchange","Expected Delivery","Availability","Other Details","Product Url"])
df           

,Brand Name,Product,Rating,No. of rating,Price,Return/Exchange,Expected Delivery,Availability,Other Details,Product Url
0,Visit the Kadence Store,"Kadence Frontier Series, Acoustic Guitar With/...",,"1,652 ratings","₹4,899.00",7 Days Replacement,--,In stock.,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the VAULT Store,Vault EA40 41 inch Premium Solid Spruce-Top Cu...,,347 ratings,--,7 Days Replacement,--,In stock.,Solid Spruce Top - Solid Wood-Top guitars prov...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Kadence Store,"Kadence Guitar Acoustica Series, 41” Jumbo Siz...",,213 ratings,"₹6,999.00",7 Days Replacement,--,In stock.,CLASSIC DESIGN ---Elegant inlay provides a cla...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Brand: Cort,Cort AD810 Dreadnought Acoustic Guitar with Gi...,,411 ratings,--,7 Days Replacement,--,Only 1 left in stock.,The Cort AD810 Dreadnought Acoustic Guitar has...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",,"17,086 ratings","₹2,090.00",7 Days Replacement,--,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
61,Visit the YAMAHA Store,"Yamaha F310 Acoustic Guitar, Tobacco Brown Sun...",,59 ratings,--,7 Days Replacement,--,In stock.,Traditional Western Body,https://www.amazon.in/Yamaha-Acoustic-Guitar-T...
62,Visit the Intern Store,INTERN 41 inches Cutaway Shape Acoustic Guitar...,--,--,--,7 Days Replacement,--,,Incredible sound: Perfect for musicians-in-the...,https://www.amazon.in/Acoustic-plectrums-stabi...
63,Visit the JUAREZ Store,JUAREZ Arpéggio 104.14 cm (41 Inch) Semi-Acous...,,13 ratings,--,7 Days Replacement,--,In stock.,"Thin Line Sleek Body, Glossy finish, Number of...",https://www.amazon.in/JUAREZ-Arp%C3%A9ggio-Sem...
64,Visit the DEVICE OF URBAN INFOTECH Store,DEVICE OF URBAN INFOTECH 360 degree Rotational...,,150 ratings,--,7 Days Replacement,--,In stock.,"【Tuning Instrument】Simplicity and accuracy, cl...",https://www.amazon.in/gp/slredirect/picassoRed...


In [7]:
driver.close()

# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [221]:
# Connecting to webdriver
driver= webdriver.Chrome(r'chromedriver.exe')
time.sleep(1)

# Opening url in Chrome browser
url='https://images.google.com/'
driver.get(url)

In [222]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Fruits' in search bar
Search.send_keys('Fruits')
time.sleep(2)

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'zgAlFc')

# Clicking search button
Search_button.click()

In [223]:
# Scrolling window using ScrollBy method from 0 pixel to 25000 pixel
driver.execute_script("window.scrollBy(0,25000)","")

In [224]:
URL=[]
images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [225]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 11:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open(r'C:\Fliprobbo\Project\Fruits'+str(i)+".jpg", "wb")
    file.write(response.content)

In [226]:
driver.close()

Searching and extracting for Cars.

In [227]:
# Connecting to webdriver
driver= webdriver.Chrome(r'chromedriver.exe')
time.sleep(1)

# Opening url in Chrome browser
url='https://images.google.com/'
driver.get(url)

In [229]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cars' in search bar
Search.send_keys('Cars')
time.sleep(2)

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'zgAlFc')

# Clicking search button
Search_button.click()

In [230]:
# Scrolling window using ScrollBy method from 0 pixel to 25000 pixel
driver.execute_script("window.scrollBy(0,50000)","")

In [231]:
URL=[]
images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [232]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 11:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open(r'C:\Fliprobbo\Project\Cars\Cars' +str(i)+ ".jpg", "wb")
    file.write(response.content)

In [233]:
driver.close()

In [234]:
# Connecting to webdriver
driver= webdriver.Chrome(r'chromedriver.exe')
time.sleep(1)

# Opening url in Chrome browser
url='https://images.google.com/'
driver.get(url)

In [235]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Machine Learning' in search bar
Search.send_keys('Machine Learning')
time.sleep(2)

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'zgAlFc')

# Clicking search button
Search_button.click()

In [236]:
# Scrolling window using ScrollBy method from 0 pixel to 25000 pixel
driver.execute_script("window.scrollBy(0,50000)","")

In [237]:
URL=[]
images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [238]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 11:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open(r'C:\Fliprobbo\Project\Machine Learning\Machine Learning'+str(i)+".jpg", "wb")
    file.write(response.content)

In [239]:
driver.close()

In [240]:
# Connecting to webdriver
driver= webdriver.Chrome(r'chromedriver.exe')
time.sleep(1)

# Opening url in Chrome browser
url='https://images.google.com/'
driver.get(url)

In [241]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Guitar' in search bar
Search.send_keys('Guitar')
time.sleep(2)

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'zgAlFc')

# Clicking search button
Search_button.click()

In [242]:
# Scrolling window using ScrollBy method from 0 pixel to 25000 pixel
driver.execute_script("window.scrollBy(0,50000)","")

In [243]:
URL=[]
images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [244]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 11:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open(r'C:\Fliprobbo\Project\Guitar\Guitar'+str(i)+".jpg", "wb")
    file.write(response.content)

In [245]:
driver.close()

In [246]:
# Connecting to webdriver
driver= webdriver.Chrome(r'chromedriver.exe')
time.sleep(1)

# Opening url in Chrome browser
url='https://images.google.com/'
driver.get(url)

In [247]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cakes' in search bar
Search.send_keys('cakes')
time.sleep(2)

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'zgAlFc')

# Clicking search button
Search_button.click()

In [248]:
# Scrolling window using ScrollBy method from 0 pixel to 25000 pixel
driver.execute_script("window.scrollBy(0,50000)","")

In [249]:
URL=[]
images= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [250]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 11:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open(r'C:\Fliprobbo\Project\Cakes\Cakes'+str(i)+".jpg", "wb")
    file.write(response.content)

In [251]:
driver.close()

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [88]:
#command to run chrome
driver4 = webdriver.Chrome(r'chromedriver.exe')

#get webpage
driver4.get("https://www.flipkart.com/")
driver4.maximize_window()

#closing pop up
pop_close = driver4.find_element("xpath","/html/body/div[2]/div/div/button").click()

# Taking input from user about product search
product = input(" Enter the name of Smartphone that has to be searched : ")

#feeding input specified by user to search menu through send keys
search_product4=driver4.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone that has to be searched : 
  @media print {
    .ms-editor-squiggles-container {
      display:none !important;
    }
  }
  .ms-editor-squiggles-container {
    all: initial;
  }oneplus nord


In [89]:
#click on the search button
search_button4= driver4.find_element(By.CLASS_NAME,'L0Z3Pu').click()

In [90]:
#fetching urls of phones coming on 1st page
page1_urls = []
urls = driver4.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [91]:
#defining  empty  list
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [92]:
#scraping data from each url of page 1
for url in page1_urls:
    driver4.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #clicking on read more button
    try:
        read_more = driver4.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #scraping brand name of phone data
    try:
        brand_tags = driver4.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #scraping phone name data
    try:
        name_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #scraping phone color data
    try:
        color_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #scraping RAM data
    try:
        ram_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #scraping ROM data
    try:
        rom_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #scraping Primary camera data
    try:                                                                                    
        pri_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #scraping secondary camera data
    try:                                                                                    
        sec_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #scraping Display size data 
    try:
        disp_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #scraping battery capacity data
    try:
        if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #scraping Price data
    try:
        price_tags = driver4.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [93]:
#placing everything into dataframe
smartphone=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

smartphone

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,16MP Front Camera,-,-,-,"₹19,995",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,627",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,601",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary Camera,-,-,-,"₹19,567",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord 2T 5G,Jade Fog,12 GB,256 GB,Primary Camera,-,-,-,"₹33,798",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
5,OnePlus,Nord 2T 5G,Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,-,-,"₹33,854",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
6,OnePlus,Nord 2T 5G,Jade Fog,8 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹28,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
7,OnePlus,Nord 2T 5G,Gray Shadow,12 GB,256 GB,Primary & Secondary Camera,-,-,-,"₹33,910",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
8,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹28,831",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
9,OnePlus,Nord 2 5G,PAC-MAN,12 GB,256 GB,Primary Camera,-,-,-,"₹37,990",https://www.flipkart.com/oneplus-nord-2-5g-pac...


In [95]:
driver4.close()

# Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [29]:
#command to run chrome
driver = webdriver.Chrome(r'chromedriver.exe')

# get webpage
driver.get("https://www.google.com/maps")
driver.maximize_window()
time.sleep(2)

In [30]:
#finding the search bar
srch_bar = driver.find_element(By.XPATH,'//input[@class="tactile-searchbox-input"]')    
srch_bar.clear()                                               
srch_bar.send_keys(input('Enter the city name: ')) 

#finding the search button and clicking
srch_button = driver.find_element(By.XPATH,'//button[@id="searchbox-searchbutton"]')       
srch_button.click()
time.sleep(3)

#getting the current url
Location=driver.current_url

try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={},longitude={}".format(lat,lon))

Enter the city name: delhi

latitude=28.6466772,longitude=76.8130571


In [31]:
driver.close()

# Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [32]:
#command to run chrome
driver = webdriver.Chrome(r'chromedriver.exe')

#get webpage
driver.get("https://trak.in/")
driver.maximize_window()
time.sleep(2)

In [33]:
#click on funding deals and go to that page
fun_btn=driver.find_element(By.XPATH,"//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(fun_btn)

In [34]:
#defining empty list
funding={}
funding["Date"]=[]
funding["Startup Name"]=[]
funding["Industry or Vertical"]=[]
funding["Sub Vertical"]=[]
funding["Location"]=[]
funding["Investor"]=[]
funding["Investment Type"]=[]
funding["Amount (in $)"]=[]

for i in range(54,57):
    driver.find_element(By.XPATH,'//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    #scraping data of Date
    date_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for date in date_tags:
        funding['Date'].append(date.text)

    # Scraping data of Startup Name
    name_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in name_tags:
        funding['Startup Name'].append(name.text)
    
    #scraping data of Industry OR Vertical
    ind_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind_tags:
        funding['Industry or Vertical'].append(n.text)
    
    #scraping data of Sub-Vertical
    sv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sv_tags:
        funding['Sub Vertical'].append(sv.text)

    #scraping data of Location
    loc_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        funding['Location'].append(loc.text)
    
    #scraping data of Investor
    inv_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        funding['Investor'].append(inv.text)
        
    #scraping data of Investment Type
    invt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        funding['Investment Type'].append(invt.text)
    #scraping data of Amount
    amt_tags = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        funding['Amount (in $)'].append(amt.text)
        
#create dataframe
df=pd.DataFrame(funding)
df

,Date,Startup Name,Industry or Vertical,Sub Vertical,Location,Investor,Investment Type,Amount (in $)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [35]:
driver.close()

# Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [82]:
#connect to webdriver
driver = webdriver.Chrome(r'chromedriver.exe')

#get webpage
driver.get("https://www.digit.in/")
driver.maximize_window()
time.sleep(2)

In [84]:
#clicking on top 10 button
driver.find_element(By.XPATH,"/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a").click()

In [86]:
product_name=[]
seller=[]
price=[]

#scraping data of product name
product_tag=driver.find_elements(By.XPATH,"//*[@id='summtable']/tbody/tr/td[1]")
for i in product_tag:
    try:
        product_name.append(i.text)
    except NoSuchElementException:
        product_name.append("-")

#scraping data of seller
seller_tag=driver.find_elements(By.CLASS_NAME,"smmerchant")
for i in seller_tag:
    try:
        seller.append(i.text)
    except NoSuchElementException:
        seller.append("-")

#scraping data of price
price_tag=driver.find_elements(By.CLASS_NAME,"smprice")
for i in price_tag:
    try:
        price.append(i.text)
    except NoSuchElementException:
        price.append("-")

#creating dataframe with data
Gaming=pd.DataFrame({"Product Name":product_name,"Seller":seller,"Price":price})
Gaming

,Product Name,Seller,Price
0,MSI Titan GT77-12UHS,N/A,N/A
1,Alienware x17 R2,Dell,"₹ 389,990"
2,Acer Predator Triton 500 SE PT516-52s,N/A,"₹ 300,000"
3,Omen by HP (16-B1371TX),Amazon,"₹ 194,000"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,N/A,"₹ 172,999"
5,MSI Delta 15 (A5EFK-083IN),N/A,"₹ 188,990"
6,Omen by HP (16-c0141AX),Amazon,"₹ 129,899"
7,Lenovo Legion 5i Pro (82RF00MGIN),N/A,"₹ 230,890"
8,Alienware m15 R5 Ryzen Edition icc-c780001win,Dell,"₹ 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Croma,"₹ 131,990"


In [87]:
driver.close()

# Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [96]:
#connect to webdriver
driver8 = webdriver.Chrome(r'chromedriver.exe')

#get webpage
driver8.get("https://www.forbes.com/billionaires/")
driver8.maximize_window()
time.sleep(2)

In [97]:
#defining empty list
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []

#scraping data of Rank
rank_tag = driver8.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")
    
#scraping data of Name
name_tag = driver8.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
#scraping data of Net Worth
netwrth_tag = driver8.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")
        
#scraping data of Age
age_tag = driver8.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")
    
#scraping data of Citizenship
cit_tag = driver8.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")
    
#scraping data of Source
src_tag = driver8.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")
    
#scraping data of Industry
ind_tag = driver8.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")

In [98]:
#length of all the coloumns
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)

(200, 200, 200, 200, 200, 200, 200)

In [99]:
#create Dataframe from data
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


# Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [105]:
#connect to webdriver
driver9 = webdriver.Chrome(r'chromedriver.exe')

#get webpage
driver9.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver9.maximize_window()
time.sleep(2)

In [106]:
#scroll the webpage
i=0
while(i<100):
    driver9.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver9.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1

In [107]:
comment = []
upvote = []
comment_time = []

#scraping data of comments
comment_tag=(driver9.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")

#scraping data of votes
upvote_tag=(driver9.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")

#scraping data of comment timings
comment_time_tag=(driver9.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")

In [108]:
#create DataFrame from data
video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video

,Comment,Upvote,Comment_ Time
0,Lose yourself in the Magic of “Love” with Sidh...,34K,1 year ago
1,"Hi. I'm an American singer, Jason Ray. The sto...",61K,1 year ago
2,Who can still listening this Masterpiece again...,156,8 days ago
3,It's 2022 still the magic of this song never b...,5,2 days ago
4,How many people suddenly remember and came her...,125,13 days ago
...,...,...,...
355,Sid is so underrated.. I wish he get some bloc...,171,1 year ago
356,This is my song,,12 days ago
357,I love this song,,2 weeks ago
358,Wow how much viral this song is,,1 month ago


In [110]:
#closing driver
driver9.close()

# Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [135]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get("https://www.hostelworld.com/")
driver.maximize_window()
time.sleep(2)

In [138]:
#select london
driver.find_element(By.ID,'location-text-input-field').click()
time.sleep(2)
driver.find_element(By.XPATH,'//*[@id="search-input-field"]').send_keys('London')
time.sleep(4)


# do click on search button
driver.find_element(By.XPATH,'//*[@id="predicted-search-results"]/li[2]').click()
driver.find_element(By.ID,'search-button').click()

In [142]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [144]:
while(True):
    
    # Hostel name
    names = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city
    distance = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for d in distance:
        Distance.append(d.text)
    time.sleep(2)
        
    #Review    
    review = driver.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total number of reviews     
    t_review = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for t in t_review:
        total_reviews.append(t.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(2) 

# Separate  Privates_From price  and  Dorms_From price
private = []
for i in range(0,len(price),2):
    private.append(price[i])
time.sleep(2)

dorms = []
for i in range(1,len(price),2):
    dorms.append(price[i]) 

In [146]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-prev"]')
        next_button.click()
    except:
        break
        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    # Property Description
    try:
        pd = driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [147]:
print(
    len(Hostel_Name),len(Distance),len(Rating),len(total_reviews),len(overall_review),
    len(private),len(dorms),len(facilities),len(property_description))

30 30 30 30 33 30 30 30 30


In [148]:
Hostel=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance":Distance,"Rating":Rating,"Total Reviews":total_reviews,"Overall Review":overall_review[:30],
                     "Private Price":private,"Dorms Price":dorms,"Facilities":facilities,"Property Description":property_description})
Hostel

,Hostel Name,Distance,Rating,Total Reviews,Overall Review,Private Price,Dorms Price,Facilities,Property Description
0,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre,No,15565 Total Reviews,Fabulous,Privates From\nRs16672,Dorms From\nRs3973,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
1,St Christopher's Village,Hostel - 1.8km from city centre,No,11479 Total Reviews,Fabulous,Privates From\nRs16900.04 Rs15210,Dorms From\nRs3396.92 Rs3057,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
2,Selina Camden,Hostel - 5.5km from city centre,No,36 Total Reviews,Fabulous,Privates From\nRs40333.38 Rs36300,Dorms From\nRs9687.38 Rs8234,Free WiFi,No Description
3,Wombat's City Hostel London,Hostel - 3.6km from city centre,No,14009 Total Reviews,Fabulous,No Privates Available,Dorms From\nRs4970.97 Rs3728,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
4,Generator London,Hostel - 3km from city centre,No,7138 Total Reviews,Fabulous,Privates From\nRs13101,Dorms From\nRs4662,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
5,Urbany Hostel London,Hostel - 5.4km from city centre,No,453 Total Reviews,Fabulous,Privates From\nRs14586,Dorms From\nRs3403,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
6,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,No,4474 Total Reviews,Superb,Privates From\nRs17404,Dorms From\nRs3895,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
7,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,No,2920 Total Reviews,Very Good,No Privates Available,Dorms From\nRs3326,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
8,No.8 Willesden Hostel London,Hostel - 10km from city centre,No,4831 Total Reviews,Superb,Privates From\nRs2942,Dorms From\nRs1745,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
9,No.8 Seven Sisters,Hostel - 9km from city centre,No,3847 Total Reviews,Very Good,No Privates Available,Dorms From\nRs1469,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
